In [ ]:
%reload_ext autoreload
%autoreload 2

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

In [ ]:
import time
import numpy as np
import pandas as pd
import plotly.express as px
from src.xzzx import XZZXDecoderBase as XZZXDecoder

In [ ]:
NUM_SHOTS_BASE = 1e2
NUM_SIZE_STEPS = 7 # 3, 5, 7, etc.
NUM_CPU_STEPS = 10 # 1, 2, 3, etc.
METHOD = "bp+mwpm+multi"
NOISE_MODEL = [0.04, 1, "Z"]
NUM_BP_PASSES = 50

time_arr = np.zeros((NUM_SIZE_STEPS, NUM_CPU_STEPS), dtype=np.float64)
res_arr = np.zeros((NUM_SIZE_STEPS, NUM_CPU_STEPS, 3), dtype=np.float64)

for s_ind in range(NUM_SIZE_STEPS):
    size = 2 * s_ind + 3
    for c_ind in range(NUM_CPU_STEPS):
        cpu = c_ind + 1
        NUM_SHOTS = int(NUM_SHOTS_BASE * (3 ** 2 / size ** 2)) * cpu
        decoder = XZZXDecoder(size, size, 1, METHOD, noise_model=NOISE_MODEL)
        decoder.run_decoder(1, 1, num_cpu=cpu, display_progress=False) # compile if possible
        start_time = time.time()
        res = decoder.run_decoder(NUM_SHOTS, NUM_BP_PASSES, num_cpu=cpu)
        end_time = time.time()
        time_arr[s_ind, c_ind] = (end_time - start_time) / NUM_SHOTS
        res_arr[s_ind, c_ind] = res

In [ ]:
tta = time_arr

In [ ]:
wc_time_df = pd.DataFrame(tta, columns=range(1, 11), index=range(3, 13, 2))

wc_time_fig = px.scatter(
    wc_time_df, log_y=True,
    title='Wall Clock Time (i7-8086K, 6C12T @ 4.4GHz)',
    labels={
        'index': 'Lattice Size d',
        'value': 'Average Time per Shot (sec)',
        'variable': '# of CPUs'
    }
).update_traces(mode='lines+markers')

wc_time_fig.write_html('perf/wc_time.html')

In [ ]:
cpu_time_df = pd.DataFrame(tta * range(1, 11), columns=range(1, 11), index=range(3, 13, 2))

cpu_time_fig = px.scatter(
    cpu_time_df, log_y=True,
    title='CPU Time (i7-8086K, 6C12T @ 4.4GHz)',
    labels={
        'index': 'Lattice Size d',
        'value': 'Average CPU Time per Shot (sec)',
        'variable': '# of CPUs'
    }
).update_traces(mode='lines+markers')

cpu_time_fig.write_html('perf/cpu_time.html')

In [ ]:
rel_perf_df = pd.DataFrame(1 / ((tta * range(1, 11)).T / tta[:, 0]).T, columns=range(1, 11), index=range(3, 13, 2))

rel_perf_fig = px.scatter(
    rel_perf_df, log_y=True,
    title='Performance Relative to Single-Core (i7-8086K, 6C12T @ 4.4GHz)',
    labels={
        'index': 'Lattice Size d',
        'value': 'Relative Performance (High Is Better)',
        'variable': '# of CPUs'
    }
).update_traces(mode='lines+markers')

rel_perf_fig.write_html('perf/rel_perf.html')